# Week 05 Homework - Corrupt Researcher
### Can knowledge and communication stop corruption?
Here is a small challenge. In a research community (e.g., uni faculty, conferences) consisting of researchers (e.g., professors, etc), every researcher knows each other. There are good researchers and a corrupt one. Each researcher knows about some other researchers and whether each of them is good or corrupt, but s/he doesn't know whether her/himself is corrupt or not. One day, a queen who has the power to know everything about all communities, came to the research community and told that "there is one corrupt researcher in this community. You should not exchange with each other what you already know about the corruption. I ask any of you to leave in the midnight of the day once you know that yourself is corrupt."
- Question: how many days will it take to get rid of the corrupt researcher? in what condition? How many possible answers? Can the community get rid of the corrupt one at all? If not, what is academia pursuing?
- Related: https://www.researchgate.net/post/Which_is_more_important_for_a_good_professor_citations_ethics_or_morals

Version 2: In a committee of $n$ researchers, each researcher interacts with exactly $k$ other researchers each day and finds out whether any of the $k$ is corrupt. The researcher then gossips the new finding with 1 other researcher on that day. Note that the corrupt research can also gossip, but his/her message can be true/wrong each time. If the queen comes and tells that there is one corrupt researcher, can the committee spot it out? how many days? if there is no such queen, can the committee still find it out?

### A few undefined parameters:

1. Is $k$ fixed for all researchers and all days, or can $k$ change every day, or differ between researchers?
1. Number of corrupt researchers?
1. Gossip
    1. Exactly what information is shared during a gossip? Possibilities:
        1. Whether or not one of $k$ is corrupt.
        1. Which researchers are included in $k$?
        1. Exactly which encountered researchers, $k_i$, are corrupt or not?
    1. What does a true/false piece of gossip (from a corrupt researcher) look like for each of the previously stated cases?
    1. One or two-way information travel?
    1. Is all knowledge shared or only daily knowledge?   

In [1]:
import numpy as np

In [2]:
class researcher:
    """
    Researcher object
    Methods:
        interact(), gossip()
    Initialized with the folloiwng inputs:
        ident (int): researcher's identity number
        corrupt (bool): whether or not researcher is corrupt
    """
    
    def __init__(self, ident, corrupt):    
        self.ident = ident
        self.corrupt = corrupt
        self.contacts = set()
        self.known_corrupt = set()
        self.known_no_corrupt = set()
        self.known_corrupt_self = False
        self.day = 0
        
    def interact(self, others):
        """
        Input:
            others (list): list of researcher objects the current
                researcher interactions with
        """
        idents = set([other.ident for other in others])
        self.today = idents
        self.today_corrupt = False
        
        for other in others:
            self.contacts.add(other.ident)
            if other.corrupt:
                self.today_corrupt = True
                self.known_corrupt.add(other.ident)
            else:
                self.known_no_corrupt.add(other.ident)

        
    def gossip(self, other):
        """
        Input:
            other (researcher): who the researcher gossips with
        """
        self.today_gossip = other.ident
        if not self.today_corrupt:
            other.known_no_corrupt.update(self.today)
            #other.contacts.update(self.today)
        

In [120]:
class institute:
    
    def __init__(self, n, nCorrupt=False):
        self.n = n
        if not nCorrupt:
            nCorrupt = np.random.randint(n+1)
        self.nCorrupt = nCorrupt
        self.nCorrupt0 = nCorrupt
        self.day = 0
        self.all_contacts_made = False
        self.all_contacts_made_day = -1
        self.no_corruption_day = -1
        
        self.corrupt = set()
        everyone = set(range(n))
        for i in range(nCorrupt):
            who = np.random.choice(list(everyone))
            self.corrupt.add(who)
            everyone.discard(who)
        
        self.researchers = []
        for i in range(n):
            if i in self.corrupt:
                self.researchers.append(researcher(i, True))
            else:
                self.researchers.append(researcher(i, False))
                
    def daily_interact(self,all_researchers):
        for r in range(len(self.researchers)):
            researcher = self.researchers[r]
            researcher.day += 1
            
            interactors = all_researchers.copy()
            interactors.remove(r)
            
            interactions = set()
            k = np.random.randint(1, self.n-1)
            for i in range(k):
                who = np.random.choice(list(interactors))
                interactions.add(who)
                interactors.remove(who)
                
            researcher.interact([self.researchers[i] for i in interactions])
            
    def daily_gossip(self,all_researchers):
        for r in range(len(self.researchers)):
            researcher = self.researchers[r]
            gossipers = all_researchers.copy()
            gossipers.remove(r)
            gossipers.difference_update(researcher.today)
            
            who = np.random.choice(list(gossipers))
            researcher.gossip(self.researchers[who])
            
    def all_contacts_made_check(self):
        if not self.all_contacts_made:
            self.all_contacts_made = True
            for r in self.researchers:
                if len(r.contacts) < self.n - 1:
                    self.all_contacts_made = False
                    break
            if self.all_contacts_made:
                self.all_contacts_made_day = self.day
        return self.all_contacts_made
                
    def leave_if_corrupt(self):
        if self.nCorrupt > 0:
            days_since_contacts_made = self.day - self.all_contacts_made_day
            for r in range(len(self.researchers)):
                if days_since_contacts_made == len(self.researchers[r].known_corrupt):
                    del self.researchers[r]
            self.nCorrupt = 0
            for r in range(len(self.researchers)):
                if self.researchers[r].corrupt:
                    self.nCorrupt += 1
            if self.nCorrupt == 0:
                self.no_corruption_day = self.day
            self.n = len(self.researchers)
            
    def daily_routine(self):
        all_researchers = set(range(len(self.researchers)))
        self.day += 1
        
        self.daily_interact(all_researchers)
        self.daily_gossip(all_researchers)
        all_contacts_made = self.all_contacts_made_check()
        if all_contacts_made:
            self.leave_if_corrupt()

In [71]:
university = institute(5)

print(university.nCorrupt)

while university.no_corruption_day == -1:
    university.daily_routine()

print(university.nCorrupt, university.all_contacts_made_day, university.no_corruption_day)
print(university.nCorrupt0, university.no_corruption_day - university.all_contacts_made_day)

5


ValueError: Range cannot be empty (low >= high) unless no samples are taken

In [121]:
university = institute(5)

#day = 0
#go_on = True
#while go_on == True:
#    go_on = False
#    for r in university.researchers:
#        if len(r.contacts) < university.n - 1:
#            go_on = True
#            break
#    university.daily_routine()
#    day += 1

In [134]:
university.daily_routine()
print('Day = ',university.day,university.all_contacts_made_day,'\n')
for r in university.researchers:
    print('identity: ',r.ident)
    print('corrupt: ',r.corrupt)
    print('contacts: ',r.contacts)
    print('known_corrupt: ',r.known_corrupt)
    print('known_no_corrupt: ',r.known_no_corrupt)
    print('today: ',r.today)
    print('today_corrupt: ',r.today_corrupt)
    print('today_gossip: ',r.today_gossip)
    print('\n')

IndexError: list index out of range

In [69]:
university.researchers